In [17]:
import os
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv()
API_URL = os.getenv("API_URL")
DATE_BEGIN = os.getenv("DATE_BEGIN")
DATE_END = os.getenv("DATE_END")

# ==== 1. Загрузка данных по API ====
visits_url = f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}"
regs_url = f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}"

# Загрузка данных
visits_df = pd.DataFrame(requests.get(visits_url).json())
regs_df = pd.DataFrame(requests.get(regs_url).json())

# ==== 2. Преобразуем данные и очищаем их ====
# Преобразуем дату для visits и registrations
visits_df["date"] = pd.to_datetime(visits_df["datetime"]).dt.date
regs_df["date"] = pd.to_datetime(regs_df["datetime"]).dt.date

# Удаляем ботов из визитов
visits_df = visits_df[~visits_df["user_agent"].str.contains("bot", case=False, na=False)]

# Группируем данные по дате и платформам
visits_grouped = visits_df.groupby(["date", "platform"]).size().reset_index(name="visits")
regs_grouped = regs_df.groupby(["date", "platform"]).size().reset_index(name="registrations")

# ==== 3. Объединяем данные по визитам и регистрациям ====
merged_df = pd.merge(visits_grouped, regs_grouped, on=["date", "platform"], how="outer")

# Заполняем пропуски
merged_df["visits"] = merged_df["visits"].fillna(0).astype(int)
merged_df["registrations"] = merged_df["registrations"].fillna(0).astype(int)

# Рассчитываем конверсию
merged_df["conversion"] = merged_df.apply(
    lambda row: (row["registrations"] / row["visits"] * 100) if row["visits"] > 0 else 0,
    axis=1
)

# ==== 4. Загружаем данные по рекламе ====
ads_id = "12vCtGhJlcK_CBcs8ES3BfEPbk6OJ45Qj"
ads_url = f"https://drive.google.com/uc?export=download&id={ads_id}"
ads_response = requests.get(ads_url, timeout=10)

# Сохраняем CSV с рекламными данными
with open("ads.csv", "wb") as f:
    f.write(ads_response.content)

ads_df = pd.read_csv("ads.csv")
ads_df["date"] = pd.to_datetime(ads_df["date"]).dt.date  # Убедимся, что дата корректно конвертируется в формат datetime

# Преобразуем дату в правильный формат для слияния
ads_grouped = ads_df.groupby(["date", "utm_campaign"]).agg(cost=("cost", "sum")).reset_index()

# Убедимся, что столбец 'cost' числовой
ads_grouped["cost"] = pd.to_numeric(ads_grouped["cost"], errors="coerce").fillna(0)

# Преобразуем 'date' в тип datetime, чтобы гарантировать совместимость типов для слияния
ads_grouped["date"] = pd.to_datetime(ads_grouped["date"])

# Преобразуем столбец 'date' в datetime для обоих датафреймов
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Объединяем рекламные данные с агрегацией по визитам и регистрациям
ads_merged = pd.merge(merged_df, ads_grouped, on="date", how="left")

# Заполняем пустые значения в 'cost' нулями
ads_merged["cost"] = ads_merged["cost"].fillna(0)

# Сортировка по дате
ads_merged = ads_merged.sort_values("date")

# Сохранение в JSON
ads_merged.to_json("ads.json", orient="columns")

# ==== 5. Построение графиков ====

# Создаём директорию для сохранения графиков
os.makedirs("./charts", exist_ok=True)
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# 📈 1. Итоговые визиты
visits_total = merged_df.groupby("date")["visits"].sum().reset_index()
sns.lineplot(data=visits_total, x="date", y="visits")
plt.title("Итоговые визиты")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/total_visits.png")
plt.close()

# 📈 2. Визиты по платформам
sns.lineplot(data=merged_df, x="date", y="visits", hue="platform")
plt.title("Визиты по платформам")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/visits_by_platform.png")
plt.close()

# 📈 3. Итоговые регистрации
regs_total = merged_df.groupby("date")["registrations"].sum().reset_index()
sns.lineplot(data=regs_total, x="date", y="registrations")
plt.title("Итоговые регистрации")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/total_registrations.png")
plt.close()

# 📈 4. Регистрации по платформам
sns.lineplot(data=merged_df, x="date", y="registrations", hue="platform")
plt.title("Регистрации по платформам")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/registrations_by_platform.png")
plt.close()

# 📈 5. Конверсия по платформам
sns.lineplot(data=merged_df, x="date", y="conversion", hue="platform")
plt.title("Конверсия по платформам")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/conversion_by_platform.png")
plt.close()

# 📈 6. Средняя конверсия
conv_avg = merged_df.groupby("date")["conversion"].mean().reset_index()
sns.lineplot(data=conv_avg, x="date", y="conversion")
plt.title("Средняя конверсия")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/average_conversion.png")
plt.close()

# 📈 7. Затраты на рекламу
ads_costs = ads_merged.groupby("date")["cost"].sum().reset_index()
sns.lineplot(data=ads_costs, x="date", y="cost")
plt.title("Затраты на рекламу")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/ads_costs.png")
plt.close()

# 📈 8. Визиты по рекламным кампаниям
sns.scatterplot(data=ads_merged, x="date", y="visits", hue="utm_campaign")
plt.title("Визиты по рекламным кампаниям")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/visits_by_campaign.png")
plt.close()

# 📈 9. Регистрации по рекламным кампаниям
sns.scatterplot(data=ads_merged, x="date", y="registrations", hue="utm_campaign")
plt.title("Регистрации по рекламным кампаниям")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./charts/registrations_by_campaign.png")
plt.close()
